In [17]:
import sys
sys.path.append('..')

from pub_crawler.tests.spiders.test_beer_spider import *

In [18]:
response = get_response(STONE_ENJOY_010122_DATA)

In [19]:
from bs4 import BeautifulSoup
import re
soup = BeautifulSoup(response.text, 'html.parser')

In [20]:
soup.find(text=re.compile("ABV"))

'ABV'

In [21]:
import scrapy

In [22]:
# request = scrapy.Request("https://ardentcraftales.com/menus")
request = scrapy.Request("https://www.stonebrewing.com/beer/year-round-releases/stone-ipa#ageGatePassed")

In [23]:
def correct_elements_exist(div):
    # Only one element per search
    name = div.find_all(class_=re.compile("name"))
    style = div.find_all(class_=re.compile("beer-style"))
    abv = div.find_all(class_=re.compile("abv"))
    only_one_element = (0 < len(elements) < 5 for elements in [abv, name, style])
    return all(only_one_element)
    # All required elements exist
    

def data_exists(div):
    name = div.find(class_=re.compile("name")).text
    style = div.find(class_=re.compile("beer-style")).text
    abv = div.find(class_=re.compile("abv")).text
    return all(name, style, abv)

In [25]:
"""
The basic idea here works if we can supplement the names for classes with different spellings AND
supplement for finding text matches as well

Some issues found here:
- The name includes the style as bs4 is printing all text contained in an element
- The search feels slow and could probably be optimized with better filtering technique
- This works with Ardent, but need to test on other sites as well before starting to commit code
- The `correct_elements_exist` filter func makes a HUGE assumption of have less that 5 elements match
"""

candidates = soup.find_all(correct_elements_exist)

data = []
for candidate in candidates:
    d = {
        "name": candidate.find(class_=re.compile("name")).text.strip(),
        "style": candidate.find(class_=re.compile("beer-style")).text.strip(),
        "abv": candidate.find(class_=re.compile("abv")).text.strip()
    }
    if d not in data:
        data.append(d)
        
data

[{'name': 'Double IPA', 'style': 'Style\nDouble IPA', 'abv': 'ABV\n9.4%'}]

In [ ]:
"""
Testing with a Stone site shows some other issues like including the "Style" and "ABV" text in the results.
Looks like result text should be split by '\n' and then keywords eliminated (i.e. delete "Style" from the array)
"""